# 1 Acessar site e coletar os PDFs

## 1.1 
pagina que hospeda os arquivos https://www.ceasa.rj.gov.br/Cota%C3%A7%C3%A3o 

In [2]:
pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 KB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup

URL_CEASA_cotacao = 'https://www.ceasa.rj.gov.br/Cota%C3%A7%C3%A3o'
urls = []

def pegar_link(url):
  """
    Função recursiva que entra no site em busca links das tags 'a' na div com id="main" e entra no novo site em busta de links. ele para quando encrontra uma url com '.pdf'.

    Args:
      url (str):endereço da pagina html a ser percorrida
        (opcional)

    Return:
      None
  """
  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')

  for link in soup.select('#main a'):
    a = str(link.get('href'))

    if ".pdf" in a:
      a = "https://www.ceasa.rj.gov.br"+a

      urls.append(a)

    else:
      pegar_link(a)

pegar_link(URL_CEASA_cotacao)


['https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20de%20pre%C3%A7os%20%2007%2011%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20de%20pre%C3%A7os%20%2006%2011%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20de%20pre%C3%A7os%20%2001%2011%202023_0.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20di%C3%A1rio%20de%20pre%C3%A7os%20%2031%2010%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20di%C3%83%C2%A1rio%20de%20pre%C3%83%C2%A7os%20%2030%2010%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20di%C3%A1rio%20de%20pre%C3%A7os%20%2027%2010%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20di%C3%A1rio%20de%20pre%C3%A7os%20%2026%2010%202023.pdf',
 'https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20di%C3%A1rio%2

### request linear

In [34]:
import re #regex
from datetime import datetime as dt
import pandas as pd


PASTA_PDFs = "./pdfs/"
PASTA_DADOS = "./dados/"


def extrair_dados_url(url):
  """
      Pega a url do PDF e extrai data, nome do documento e link do pdf

    Args:
      link: str
        url do documento pdf

    Return:
      url: str
      nome_arquivo: str
      data: str
  """
  # pega nome do arquivo na URL
  nome_arquivo = url.split("/")[-1]
  nome_arquivo = requests.utils.unquote(nome_arquivo)

  # padrao de dd mm yyyy para data
  matches = re.findall(r'\b(\d{1,2})\s(\d{1,2})\s(\d{4})\b', nome_arquivo)
  data = dt.strptime("/".join(matches[0]), "%d/%m/%Y")

  return url,"CEASA-RJ_" + nome_arquivo.replace(" ", "_") ,data.strftime("%d-%m-%Y")

def pegar_pdf(url,nome_arquivo):
  response = requests.get(url)

  if response.status_code == 200:
    # lista_url[i] = [lista_url[i],nome_arquivo]
    with open(PASTA_PDFs+nome_arquivo, "wb") as f:
      f.write(response.content)
      return True

def add_csv(url, nome_arquivo,data):
  """
  Gera um arquivo relativo a coleta de pdfs

  Args:
    Tupla: tupla
      recebe uma tupla com url, nome do arquivo e data
  """

  # nome dos arquivos
  with open(PASTA_DADOS+"ceasa_lista_pdf.csv", "a") as f:
    if url is None:
      f.write("URL;Nome_Arquivo;Data\n")
    else:
      f.write(url)
      f.write(";")
      f.write(nome_arquivo )
      f.write(";")
      f.write(str(data))
      f.write('\n')


for x in urls[:2]:
  metadados = extrair_dados_url(x)
  print(metadados)
  url,nome_arquivo,data = metadados
  pegar_pdf(url,nome_arquivo)
  add_csv(url,nome_arquivo,data)


# pegar_link()

# print(extrair_dados_url("https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20dia%CC%81rio%20de%20prec%CC%A7os%20%2002%2006%202023.pdf"))


('https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20de%20pre%C3%A7os%20%2007%2011%202023.pdf', 'CEASA-RJ_Boletim_de_preços__07_11_2023.pdf', '07-11-2023')
('https://www.ceasa.rj.gov.br/sites/default/files/arquivos_paginas/Boletim%20de%20pre%C3%A7os%20%2006%2011%202023.pdf', 'CEASA-RJ_Boletim_de_preços__06_11_2023.pdf', '06-11-2023')


### request async

In [35]:
pip install aiohttp

22418.71s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [43]:
import re #regex
from datetime import datetime as dt
import pandas as pd

import aiohttp
import asyncio



PASTA_PDFs = "./pdfs/"
PASTA_DADOS = "./dados/"


def extrair_dados_url(url):
  """
      Pega a url do PDF e extrai data, nome do documento e link do pdf

    Args:
      link: str
        url do documento pdf

    Return:
      url: str
      nome_arquivo: str
      data: str
  """
  # pega nome do arquivo na URL
  nome_arquivo = url.split("/")[-1]
  nome_arquivo = requests.utils.unquote(nome_arquivo)

  # padrao de dd mm yyyy para data
  matches = re.findall(r'\b(\d{1,2})\s(\d{1,2})\s(\d{4})\b', nome_arquivo)
  data = dt.strptime("/".join(matches[0]), "%d/%m/%Y")

  return url,"CEASA-RJ_" + nome_arquivo.replace(" ", "_") ,data.strftime("%d-%m-%Y")

async def pegar_pdf(resp, url,nome_arquivo):
  response = await resp

  if response.status_code == 200:
    # lista_url[i] = [lista_url[i],nome_arquivo]
    with open(PASTA_PDFs+nome_arquivo, "wb") as f:
      f.write(response.content)
      return True

async def add_csv(url, nome_arquivo,data):
  """
  Gera um arquivo relativo a coleta de pdfs

  Args:
    Tupla: tupla
      recebe uma tupla com url, nome do arquivo e data
  """

  # nome dos arquivos
  with open(PASTA_DADOS+"ceasa_lista_pdf.csv", "a") as f:
    if url is None:
      f.write("URL;Nome_Arquivo;Data\n")
    else:
      f.write(url)
      f.write(";")
      f.write(nome_arquivo )
      f.write(";")
      f.write(str(data))
      f.write('\n')

# async orquestra

async def requsicao_API(session, url):
    async with session.get(url,timeout=60*100) as resp:
        
        try:
            metadados = extrair_dados_url(x)
            url,nome_arquivo,data = metadados
            pegar_pdf(resp,url,nome_arquivo)
            add_csv(url,nome_arquivo,data)
                        
        except Exception as e:
            await print("requsicao_API", e,url)   


async def main():
  
  urls = urls[:2]

  session_timeout =   aiohttp.ClientTimeout(total=None,sock_connect=20 ,sock_read=20)
  connector = aiohttp.TCPConnector(limit_per_host=5)
  async with aiohttp.ClientSession(headers=None,timeout=session_timeout, connector=connector) as session:
      tasks = []      
      
      for url in urls:
          tasks.append(asyncio.ensure_future(requsicao_API(session, url)))

      try:
          await asyncio.gather(*tasks)

      except Exception as e:
          await print("main",e,'url')



# for x in urls[:2]:
#   metadados = extrair_dados_url(x)
#   print(metadados)
#   url,nome_arquivo,data = metadados
#   pegar_pdf(url,nome_arquivo)
#   add_csv(url,nome_arquivo,data)



# asyncio.run(main())
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

RuntimeError: This event loop is already running